## PW on graphcut optimization (binary case)
This session is divided into 3 parts:


* a part on Bayesian classification
* a part on object/background segmentation of a colour image with a CRF (conditional random filed)
* a part on the iterative segmentation of a textured image

We will use the PyMaxflow library for the calculation of the graphcut.

In [ ]:
# -*- coding: utf-8 -*-
"""TP_graphcut_part_1.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/10oEfwo5gV-LTy9YZIzd4IqwuiF4ROaCW

## PW on graphcut optimization (binary case)
This session is divided into 3 parts:
* a part on Bayesian classification
* a part on object/background segmentation of a colour image with a CRF (conditional random filed)
* a part on the iterative segmentation of a textured image

We will use the PyMaxflow library for the calculation of the graphcut.
"""
import numpy as np
import matplotlib.pyplot as plt
import requests
import imageio
import scipy.signal
import scipy.ndimage

from io import BytesIO
from PIL import Image
from skimage.morphology import binary_dilation, disk
from skimage import color
from sklearn.cluster import KMeans

try:
    import maxflow  # if not installed, install Maxflow
except:
    !pip install PyMaxflow # For Google Collab
    import maxflow

from bokeh.plotting import figure
from bokeh.plotting import show as showbokeh
from bokeh.io import output_notebook

output_notebook()

def affiche_pour_colab(im, MINI=None, MAXI=None, titre=""):  # special colab, don't look
    """
    Display an image in a Jupyter notebook using Bokeh.

    Args:
        im (ndarray): The input image.
        MINI (float): The minimum value for normalization (optional).
        MAXI (float): The maximum value for normalization (optional).
        titre (string): The title of the plot (optional).
    """

    p = figure(
        tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
        y_range=[im.shape[0], 0],
        x_range=[0, im.shape[1]],
        title=titre,
    )
    # p.x_range.range_padding = p.y_range.range_padding = 0
    # must give a vector of images
    p.image(
        image=[np.flipud(im)],
        x=0,
        y=im.shape[0],
        dw=im.shape[1],
        dh=im.shape[0],
        palette="Greys9",
        level="image",
    )
    p.xgrid.visible = False
    p.ygrid.visible = False
    showbokeh(p)

def affiche(im, MINI=0.0, MAXI=None, titre="", printname=False):
    """
    Display an image in a Jupyter notebook using Bokeh.

    Args:
        im (ndarray): The input image.
        MINI (float): The minimum value for normalization (optional).
        MAXI (float): The maximum value for normalization (optional).
        titre (string): The title of the plot (optional).
        printname (boolean): Whether to print the name (optional).
    """
    affiche_pour_colab(
        im, MINI=MINI, MAXI=MAXI, titre=titre
    )  # under google colab many options disappear

def display_segmentation_borders(image, bin):
    """
    Display the segmentation borders on the image.

    Args:
        image (ndarray): The input image.
        bin (ndarray): The binary segmentation mask.

    Returns:
        numpy.ndarray: The image with segmentation borders highlighted.
    """
    contour = binary_dilation(bin, disk(15)) ^ bin

    imagergb = np.copy(image)
    imagergb[contour == 1, 0] = 255
    imagergb[contour == 1, 1] = 0
    imagergb[contour == 1, 2] = 0

    return imagergb

## Binary classification of a noisy image

You have a binary image *IoriginaleBW.png* (binary image of the two classes) and its observed version with a certain distribution of grey levels for each class *Iobservee.png*. The objective is to perform a two-class classification of this observed image (see PW1 and PW2).

### Analysis of the distributions of the 2 classes of the image}


Q1: What are the distributions of the two classes of the image ($P(Y_s|X_s=0)$ (black class) and $P(Y_s|X_s=0)$ (white class))?

Q2: Give the means and variances of the two classes.

*The distributions and the means and variances found in the previous sessions will be used without justification*.

**Your answer &#x270D;**

A1: Both classes follow a normal distribution.

A2: Class 0 : mean = 97.4, variance = 22.3  
    Class 1 : mean = 163.9, variance = 22.7

## 1.1: Graphcut optimization

Q3: How many nodes does the graph have that is constructed for the search for the minimum capacity cut with only two neighbouring pixels? What do they correspond to? What do the data attachment terms in this graph correspond to and what values do they have for two observed pixels of values $y_s$ and $y_t$? What does the regularisation term correspond to?

**Your answer &#x270D;**

A3: This graph has 4 nodes. Two of them correspond to the pixels. The others correspond to the labels (one for each class). The attachment terms in this graph are the edges between the label nodes and the pixel nodes. The value of each of these links is equal to the negative log-likelihood of the value of the pixel node with respect to the corresponding classes distribution.  
With $\mu_i$ the mean of the pixels' distributions, the edges' weights are $(y_s-\mu_i)²$
The regularisation terms are equal to $\beta$ and correspond to the edges between the pixel nodes.


Q4: Complete the python code cell where it says "#TO BE COMPLETED EX1" with the data attachment and regularization terms as indicated. Run the minimum cut algorithm and view the result.

## 1.2 Searching for the optimal $\beta$.



Q5: By completing the program frame provided below, find the optimal $\beta$ value $\beta_{opt}$ using the "true image" $x$ corresponding to IoriginaleBW.png. You can plot the error values between $x$ and the estimated $\hat{x}$ to find $\beta_{opt}$.



**Your answer &#x270D;**

A5: $\beta_{opt}$ = 1890

Q6: What are the advantages of this optimization approach compared to ICM? Compared to simulated annealing? In theory, do we obtain the same result with both methods (simulated annealing and graphcut)? Under what conditions? What is the advantage of simulated annealing in the general case?

**Your answer &#x270D;**

A6: The advantages of this optimization approach compared to ICM are that the solution found by the graph cut is a global minimum whereas ICM finds a local minimum. Graph cut's solution is thus better. Also, we find the solution is found faster with the graph cut method. The ICM method also requires an initialisation while graph cut doesn't.



The main advantage of graph cut over simulated annealing is that it finds the solution much faster. Like ICM, simulated also requires an initialisation that graph cut doesn't.



In theory, we obtain the same result with both methods under the condition that we decrease the temperature slowly enough.


In practice, the simulated annealing always gives the global minimum no matter the model while graph cut finds the global minimum considering the model used.


Q7: How can you explain that the error rate with the true image can be lower with the simulated annealing result or the ICM than with the graph-cut optimisation?
  

**Your answer &#x270D;**

A7: We don't want to choose the global minimum found by the graph cut because the latter doesn't model the problem perfectly whereas the solution found with simulated annealing is better in the sense that it's a stochastic approximation of the global minimum and is therefore more realistic.

Q8: What are the advantages and disadvantages of the Ising model?

**Your answer &#x270D;**

A8: The Ising model is interesting because it only involves a close neighborhood of the pixels we are looking at. This is computationaly interesting. Also we often find in natural images that neighboring pixels have very similar colors or intensities.
However the problem could be that thin structures that involve a line of pixels being correlated instead of the correlation of all the nearest neighbors can be easily ignored. For example, when looking at a vein pixel, the surrounding ones can be part of the background and the Ising model will push to make the vein pixel background. Another disadvantage of the Ising model is that it is too restrictive if we decide to apply it on grayscale images. In this case, this model is called Potts.

In [ ]:
# Loading images

def open_image_from_url(url):
    """
    Opens an image from the given URL and returns it as a NumPy array.

    Args:
        url (str): The URL of the image to be opened.

    Returns:
        numpy.ndarray: The image as a NumPy array.
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    return np.array(img)

# Observed image, noisy
im_obs = (
    open_image_from_url("https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee.png")
    * 255
)

# Binary reference image, to assess the quality of the segmentation
im_orig = open_image_from_url(
    "https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/IoriginaleBW.png"
)

affiche(im_obs, titre="Observed image")

In [ ]:
#### to be completed
# TO BE COMPLETED EX1
beta = 1890  # beta value
m0 = 97.4  # m0 and m1 values from the previous practical work
m1 = 163.9

## binary graph-cut

# Create the graph.
g = maxflow.Graph[float]()  # Graph instantiation

# Add the nodes.
# nodeids has the identifiers of the nodes in the grid.
# It creates a set of nodes for all the pixels of the image
nodeids = g.add_grid_nodes(im_obs.shape)

# Add non-terminal edges with the same capacity.
# The edge has the value beta for all adjacent pixels in 4-connexity
g.add_grid_edges(nodeids, beta)

# Add the terminal edges.
# The second argument correspond to the set of edge values to the source
# The third argument correspond to the set of edge values to the sink
g.add_grid_tedges(nodeids, (im_obs - m0) ** 2, (im_obs - m1) ** 2)

# Find the maximum flow.
flow = g.maxflow()

print("Max Flow:", str(flow))
# Get the labels of the nodes in the grid.
# Output is 0 if the node is connected to the source, else output is 1
sgm = g.get_grid_segments(nodeids)
im_bin = np.int_(np.logical_not(sgm))

affiche(im_bin, titre="Result for beta = " + str(beta))

In [ ]:
# Compute the error image between im_bin and im_orig (the ideal solution) using np.abs and np.sum
error = np.sum(im_bin != im_orig)

# Visualize the differences between the original image and the solution
plt.figure()
plt.imshow(np.dstack((np.int_(im_orig), im_bin, im_bin)) * 255)
plt.title("Result for beta = " + str(beta) + " and truth")
plt.show()

print("Number of misclassified pixels for beta = ", beta, ": ", int(error))

### Search for the best parameter $\beta$

In [ ]:
im_obs = (
    open_image_from_url("https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee.png")
    * 255
)  # Observed image, noisy
im_orig = open_image_from_url(
    "https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/IoriginaleBW.png"
)  # Binary reference image, to assess the quality of the segmentation

# TO BE COMPLETED - choose a range of values and a step to study beta
def find_beta(start, stop, step):
    list_beta = []
    list_errors = []
    for beta in range(start, stop, step):
        # TO BE COMPLETED
        m0 = 97.4
        m1 = 163.9

        ## Binary graph cut

        # Create the graph
        g = maxflow.Graph[float]()  # graph instantiation

        # Add the nodes. nodeids has the identifiers of the nodes in the grid.
        nodeids = g.add_grid_nodes(im_obs.shape)
        # Add non-terminal edges with the same capacity.
        g.add_grid_edges(nodeids, beta)
        # Add the terminal edges.
        g.add_grid_tedges(nodeids, (im_obs - m0) ** 2, (im_obs - m1) ** 2)

        # Find the maximum flow.
        g.maxflow()

        # Get the segments of the nodes in the grid.
        sgm = g.get_grid_segments(nodeids)
        # create the output image
        im_bin = np.int_(np.logical_not(sgm))

        # Compute the error
        error = np.sum(im_bin != im_orig)
        list_beta.append(beta)
        list_errors.append(error)

    plt.figure()
    plt.scatter(list_beta, list_errors)
    plt.xlabel("beta")
    plt.ylabel("number of misclassified pixels")
    plt.show()

    best_beta = list_beta[np.argmin(np.array(list_errors))]

    print("Best beta value: ", best_beta)

find_beta(1000, 3000, 100)
find_beta(1700, 1950, 5)

## 2. Classification of a colour image

The objective of this part is to carry out an extension of the method seen previously in the case of the treatment of a colour image *avions.png* in which one wants to separate the objects from the background.

We will first use the same framework (Ising model) as before but with a three-dimensional data attachment (assuming convariance matrices equal to the identity). Then we will introduce a CRF (conditional random field) by weighting the regularisation term of the Ising model by the modulus of the gradient between two pixels of the observed image.  

### 2.1 Binary classification
From the program structure below, carry out the different steps necessary for this classification:
1. Modelling of the background and object distributions (in 3 dimensions this time)
1. Definition of the data attachment term
1. Choice of a value for the regularisation parameter for the Ising model
1. Finding the minimal cut to obtain the object/background classification.

Q9: Comment on these steps and the results obtained.

**Your answer &#x270D;**

A9:

In [ ]:
# Loading and displaying the image
im_obs = open_image_from_url(
    "https://www.dropbox.com/s/ylm0ut8ipu5oonb/avions.png?dl=1"
)

plt.figure()
plt.rcParams["figure.figsize"] = [15, 15]
plt.imshow(im_obs)
plt.show()

# Conversion of the image between 0 and 255
im_obs = 255 * im_obs

#### Determining the parameters of the classes


Plane class: we can use the values of the rectangle [180:200,280:300].

example np.mean(image[180:200,280:300,1]) returns the average of the selected area for channel 1

In [ ]:
# Mean of the plane class - 3D vector
m_planes = np.mean(im_obs[180:200, 280:300], axis=(0, 1))

# Mean of the sky class
# You can use values in the following square [0:100,150:300]
m_sky = np.mean(im_obs[0:100, 150:300], axis=(0, 1))

# Check that the obtained values are coherent
print("For the sky, [R,G,B] = ", m_sky)
print("For the planes, [R,G,B] = ", m_planes)

In [ ]:
# Choose a beta value
beta = 20

# TO BE COMPLETED
## Binary graph-cut
# use the previous program to create the graph and compute the cut
# be careful of computing the terminal weights using the 3D values
# you can compute 2 distance images with, in each pixel, the quadratic distance to the mean value
# of each class
m0 = m_planes  # m0 and m1 values from the previous practical work
m1 = m_sky

## Binary graph-cut

# Create the graph.
g = maxflow.Graph[float]()  # Graph instantiation

# Add the nodes.
# nodeids has the identifiers of the nodes in the grid.
# It creates a set of nodes for all the pixels of the image
nodeids = g.add_grid_nodes(im_obs.shape)

# Add non-terminal edges with the same capacity.
# the edge has the value beta for all adjacent pixels in 4-connexity
g.add_grid_edges(nodeids, beta)

# Add the terminal edges.
# The second argument correspond to the set of edge values to the source
# The third argument correspond to the set of edge values to the sink
g.add_grid_tedges(nodeids, (im_obs - m0) ** 2, (im_obs - m1) ** 2)

# Find the maximum flow.
flow = g.maxflow()

print("Max Flow:", str(flow))
# Get the labels of the nodes in the grid.
# output is 0 if the node is connected to the source, else output is 1
sgm = g.get_grid_segments(nodeids)
im_bin = np.int_(np.logical_not(sgm))

plt.imshow(255 * im_bin)
plt.show()

print("Class is chosen by majority voting:")
im_bin = np.sum(im_bin, axis=2) // 2
plt.imshow(255 * im_bin, cmap="gray")
plt.show()

### 2.2 Use of a CRF (Conditional Random Field) model

We will try here to adapt the model used previously to favour transitions where they are compatible with the gradient. To do this, we will replace the constant $\beta$ for the whole image by a "beta_field" which depends on the norm of the gradient.

Q10: Calculate and display the modulus of the gradient of the aircraft image after it has been grayscaled and convolved by a Gaussian kernel of standard deviation 1. Why use the "boundary='symm'" option when convolving through the Sobel filter? Try it without doing the Gaussian filtering. What is the point?

**Your answer &#x270D;**

A10: In order to keep our image size, we need to add pixels around the image before convolving. To make sure these pixels make sense, we use the 'symm option because there only is sky in the borders of our image. This element is plain and can be mirrored without altering its sense.

Without Gaussian filtering, we note that the planes are almost undistinguishable against the noise. The filter reduces the noise and it allows the gradient to put the emphasis on the planes' contours.

In [ ]:
def gradient(image):
    """
    Calculate the gradient of an image using the Sobel operator.

    Args:
        image (ndarray): The input image.

    Returns:
        tuple of ndarray: The gradient of the image in the x and y directions.
    """
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float)
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float)

    # to be completed
    # use mode = 'same' and boundary='symm' in scipy.signal.convolve2d
    derivative_x = scipy.signal.convolve2d(image, sobel_x, mode="same", boundary="symm")
    derivative_y = scipy.signal.convolve2d(image, sobel_y, mode="same", boundary="symm")
    return [derivative_x, derivative_y]

plane_nb = color.rgb2gray(im_obs)
plane_x, plane_y = gradient(plane_nb)

# Calculation of the gradient modulus
grad_av = np.sqrt(np.square(plane_x) + np.square(plane_y))
plt.figure()
plt.title("Gradient without Gaussian filtering", fontsize=20)
plt.imshow(grad_av)
plt.show()

plane_nb = scipy.ndimage.gaussian_filter(color.rgb2gray(im_obs), 0.5)
plane_x, plane_y = gradient(plane_nb)

# Calculation of the gradient modulus
grad_av = np.sqrt(np.square(plane_x) + np.square(plane_y))
plt.figure()
plt.title("Gradient with Gaussian filtering", fontsize=20)
plt.imshow(grad_av)
plt.show()


Complete the code for segmentation by CRF. We will choose $beta\_field=\beta_2\cdot\exp(-grad\_av/h)$ We can use $h=300$ and $\beta_2=20000$. Also replace the constant beta by the field "beta_field" during the "g.add_grid_edges" step.

Q11: Compare the results with and without the contour term.

**Your answer &#x270D;**

A11: beta_field is used to guide the segmentation of the planes. When it is used, the borders are more regular. If the parameters are not set correctly, we find that the segmentations are too simple as we can see in the following images. When lowering the standard deviation of the Gaussian filter, we find more details but at the cost of bigger noise on the planes' contours.

In [ ]:
# Calculation of beta_field
# This field will set the value for the 4-neighbours
h = 300
beta2 = 200
beta_field = beta2 * np.exp(-grad_av / h)

beta_field = np.repeat(
    np.reshape(beta_field, (beta_field.shape[0], beta_field.shape[1], 1)), 3, axis=2
)

## Binary graph cut
# Complete by taking your previous code and replacing
# Beta by beta_field in the line g.add_grid_edges

# m0 and m1 values from the previous practical work
m0 = m_planes
m1 = m_sky

## Binary graph-cut

# Create the graph.
g = maxflow.Graph[float]()  # Graph instantiation

# Add the nodes.
# nodeids has the identifiers of the nodes in the grid.
# It creates a set of nodes for all the pixels of the image
nodeids = g.add_grid_nodes(im_obs.shape)

# Add non-terminal edges with the same capacity.
# The edge has the value beta for all adjacent pixels in 4-connexity
g.add_grid_edges(nodeids, beta_field)

# Add the terminal edges.
# The second argument correspond to the set of edge values to the source
# The third argument correspond to the set of edge values to the sink
g.add_grid_tedges(nodeids, (im_obs - m0) ** 2, (im_obs - m1) ** 2)

# Find the maximum flow.
flow = g.maxflow()

print("Max Flow:", str(flow))

# Get the segments of the nodes in the grid.
sgm = g.get_grid_segments(
    nodeids
)  # Returns 1 if the pixel is on the drain side after calculating the min cut, 0 if it is on the source side

im_bin = np.int_(np.logical_not(sgm))

# affiche(im_bin, titre="Result for the CRF model")
plt.imshow(255 * im_bin)
plt.show()

print("Class is chosen by majority voting:")
im_bin = np.sum(im_bin, axis=2) // 2
plt.imshow(255 * im_bin, cmap="gray")
plt.show()

## 3. Iterative Segmentation with Gaussian Mixture

Q12: Display the "zebra" image below. Is it possible to segment the zebra with the method used to segment the planes?



**Your answer &#x270D;**

A12: We can't use the same algorithm because the zebra's fur is made of two very contrasted colors that will be segmented from one another. The intensity of the green of the grass around the animal falls between the intensities of the black and white of the stripes.

In [ ]:
### Loading a new image
I_zebra = imageio.imread(
    "https://upload.wikimedia.org/wikipedia/commons/6/60/Equus_quagga.jpg"
)
I_zebra = I_zebra[200:, :, :]
rect_zebra = I_zebra[700:1100, 1000:2100]
rect_background = I_zebra[:, 0:600]

plt.figure()
plt.imshow(I_zebra)
plt.title("I_zebra")
plt.show()

plt.figure()
plt.imshow(rect_zebra)
plt.title("rect_zebra")
plt.show()

plt.figure()
plt.imshow(rect_background)
plt.title("rect_background")
plt.show()

Q13: Calculate the covariance matrix for the rect_zebra and the rect_background. Display the values and comment on the result. What do the diagonal values correspond to?

**Your answer &#x270D;**

A13: The diagonal values correspond to each color's variance. We find that the zebra's rectangle has very high variances which make sense since it is made of black and white, which are opposite intensities, in similar quantities. As for the background rectangle, the variances are pretty low which is explained by the fact that the patch is pretty homogeneous and fully green.

In [ ]:
V_tulip = np.vstack([I_zebra[:, :, i].flatten() for i in range(3)])

M_cov = np.cov(np.vstack([I_zebra[:, :, i].flatten() for i in range(3)]))
print("Full zebra image")
print(M_cov)

M_cov = np.cov(np.vstack([rect_zebra[:, :, i].flatten() for i in range(3)]))
print("Rectangle zebra")
print(M_cov)

M_cov = np.cov(np.vstack([rect_background[:, :, i].flatten() for i in range(3)]))
print("Rectangle background")
print(M_cov)

Q14: Display the histogram for the R, G and B channels for the "rect_zebra" image. Comment on the result.

**Your answer &#x270D;**

A14: It is fairly obvious that for each color, the histograms are very similar. We expected this result since the colors are mainly black and white which are an even intensity mix of the three colors. Moreover, we see a peak in the low intensities, which would correspond to the black parts of the image, as well as a plateau in the high intensity which would correspond to the white parts. The white part is not a peak because it is much more subject to the shade and thus a bigger span of intensities.

In [ ]:
# Allows you to change the display size of plt.show
plt.rcParams["figure.figsize"] = [11, 11]

nomchan = ["red", "green", "blue"]
for chan in range(3):
    plt.figure()
    plt.hist(rect_zebra[:, :, chan].flatten(), 100)
    plt.title("Histogram of the zebra class for the channel " + nomchan[chan])
    plt.show()

for chan in range(3):
    plt.figure()
    plt.hist(rect_background[:, :, chan].flatten(), 100)
    plt.title("Histogram of the background class for the channel " + nomchan[chan])
    plt.show()

Q15: Propose an algorithmic method to identify the two classes of the image "rect_zebra". Use a sklearn implementation of this algorithm to identify the mean vectors ($m_R$,$m_G$,$m_B$) for two classes of the "rect_zebra" image and two classes of the "rect_background" image.

*The following line of code can be used to transform the image into a suitable form.*

X = np.vstack([rect_zebra[:,:,i].flatten() for i in range(3)]).transpose()

Comment on the average vectors obtained. They can be displayed as an image using the code provided.

**Your answer &#x270D;**

A15: It is interesting to use the K-means algorithm because for each class, the variances of each color are pretty close which satisfies the hypothesis of this algorithm that the clusters are spherical. Using K=2 for rect_zebra makes sense since we observed two different stacks in the histograms.

The vectors obtained make sense since we obtain a dark color, almost black, that corresponds to the black stripes which caused the big peak in the histogram. The second vector is a pretty dark shade of white which we expected since there are many different shades in the rectangle. Also, the high intensities in the histogram showed a plateau that wasn't much higher than the other intensities which means that the mean was pulled toward darker intensities.

In [ ]:
# Computation of class parameters in a semi-automatic way

# Use a number of classes =2 for the zebra part (complete n_clusters=?)
X = np.vstack([rect_zebra[:, :, i].flatten() for i in range(3)]).transpose()
kmeans_zebra = KMeans(n_clusters=2, random_state=0).fit(X)
mean_vectors_zebra = kmeans_zebra.cluster_centers_

# Use a number of classes =2 for the background part
X = np.vstack([rect_background[:, :, i].flatten() for i in range(3)]).transpose()
kmeans_background = KMeans(n_clusters=2, random_state=0).fit(X)
mean_vectors_background = kmeans_background.cluster_centers_

In [ ]:
# Display of the class centers found as an image
image_mean_vectors = np.zeros((200, 200, 3), np.uint8)

for i in range(3):
    image_mean_vectors[0:99, 0:99, i] = mean_vectors_zebra[0, i]
    image_mean_vectors[0:99, 100:199, i] = mean_vectors_zebra[1, i]
    image_mean_vectors[100:199, 0:99, i] = mean_vectors_background[0, i]
    image_mean_vectors[100:199, 100:199, i] = mean_vectors_background[1, i]

plt.figure()
plt.imshow(image_mean_vectors, vmax=255)
plt.title("Top: zebra classes, bottom: background classes")
plt.show()

Q16: Display separately two neg-log-likelihood images (the neg-log-likelihood was used as a data attachment in the previous example) for the two zebra classes.
Build a neg-log-likelihood image corresponding to the minimum of the two neg-log-likelihoods of the zebra classes. Similarly for the background class.

Display them and comment.

**Your answer &#x270D;**

A16: For both the background and zebra classes, we can see that all of the pixels classified by the first sub-class are correct but it also misses many of its class's pixels. However, the second sub-class classifies most of the missing pixel but it also takes many pixels that are wrongly classified. In the combined images of both sub-classes, we can see some overlap even though the aimed classes are fully represented.

In [ ]:
# Calculation of the neg-log-likelihood of the zebra class
neg_log_likelihood_zebra_0 = sum(
    (I_zebra[:, :, i] - mean_vectors_zebra[0, i]) ** 2 for i in range(3)
)

plt.figure()
plt.imshow(neg_log_likelihood_zebra_0, cmap="gray", vmax=600)
plt.title("neg_log_likelihood_zebra_0")
plt.show()

neg_log_likelihood_zebra_1 = sum(
    (I_zebra[:, :, i] - mean_vectors_zebra[1, i]) ** 2 for i in range(3)
)

plt.figure()
plt.imshow(neg_log_likelihood_zebra_1, cmap="gray", vmax=5000)
plt.title("neg_log_likelihood_zebra_1")
plt.show()

neg_log_likelihood_zebra_combined = np.minimum(
    sum((I_zebra[:, :, i] - mean_vectors_zebra[0, i]) ** 2 for i in range(3)),
    sum((I_zebra[:, :, i] - mean_vectors_zebra[1, i]) ** 2 for i in range(3)),
)

plt.figure()
plt.imshow(neg_log_likelihood_zebra_combined, cmap="gray", vmax=5000)
plt.title("neg_log_likelihood_zebra_combined")
plt.show()

# TO BE COMPLETED
# calculate the neg-log-likelihood of the background
# call the output neg_log_likelihood_background_combined
neg_log_likelihood_background_0 = sum(
    (I_zebra[:, :, i] - mean_vectors_background[0, i]) ** 2 for i in range(3)
)

plt.figure()
plt.imshow(neg_log_likelihood_background_0, cmap="gray", vmax=600)
plt.title("neg_log_likelihood_background_0")
plt.show()

neg_log_likelihood_background_1 = sum(
    (I_zebra[:, :, i] - mean_vectors_background[1, i]) ** 2 for i in range(3)
)

plt.figure()
plt.imshow(neg_log_likelihood_background_1, cmap="gray", vmax=5000)
plt.title("neg_log_likelihood_background_1")
plt.show()

neg_log_likelihood_background_combined = np.minimum(
    sum((I_zebra[:, :, i] - mean_vectors_background[0, i]) ** 2 for i in range(3)),
    sum((I_zebra[:, :, i] - mean_vectors_background[1, i]) ** 2 for i in range(3)),
)

plt.figure()
plt.imshow(neg_log_likelihood_background_combined, cmap="gray", vmax=5000)
plt.title("neg_log_likelihood_background_combined")
plt.show()

Q17: From these combined data attachment images (one for the background and one for the zebra), set up a graph-cut segmentation of the image with the $\beta$ of your choice. Comment the result.

**Your answer &#x270D;**

A17: The result is pretty good but there are some major mistakes such as the zebra's behind and belly. We also see that it struggles to follow the contour around its neck and mane.

In [ ]:
beta = 100000  # Optimal beta value to be determined

## Binary graph cut

# Create the graph.

# Graph instantiation
g = maxflow.Graph[float]()
# Add the nodes. nodeids has the identifiers of the nodes in the grid.
nodeids = g.add_grid_nodes(
    I_zebra.shape[0:2]
)  # Create a grid with a non-terminal node for each pixel in the image

# Add non-terminal edges with the same capacity.
g.add_grid_edges(
    nodeids, beta
)  # Addition of a beta weight edge between each adjacent node according to the 4-connexity

# Add the terminal edges.
# TO BE COMPLETED
g.add_grid_tedges(
    nodeids, neg_log_likelihood_zebra_combined, neg_log_likelihood_background_combined
)

# Find the maximum flow.
flow = g.maxflow()

print("Max Flow:", str(flow))

# Get the segments of the nodes in the grid.
sgm = g.get_grid_segments(nodeids)
im_bin = np.int_(np.logical_not(sgm))

plt.figure()
plt.imshow(im_bin)
plt.title("Result of the segmentation")
plt.show()

plt.figure()
plt.imshow(display_segmentation_borders(I_zebra, im_bin))
plt.title("Red segmentation contours")
plt.show()

Q18: From the obtained segmentation, determine the mean vectors for 5 classes for the background and 5 classes for the zebra. Use these lists of mean vectors to construct new neg-log-likelihood combination images. Comment on them.

**Your answer &#x270D;**

A18: This time, we see that there is much less overlap between the classes. Some stripes of the zebra are classified as background but they are also classified as zebra so we can hope to segment them as zebra with a good tuning of beta.

In [ ]:
# The computing of this cell can take several minutes.
X_1 = np.vstack([I_zebra[im_bin == 1, i].flatten() for i in range(3)]).transpose()
kmeans_zebra2 = KMeans(n_clusters=5, random_state=0).fit(X_1)
mean_vectors_zebra2 = kmeans_zebra2.cluster_centers_

X_2 = np.vstack([I_zebra[im_bin == 0, i].flatten() for i in range(3)]).transpose()
kmeans_background2 = KMeans(n_clusters=5, random_state=0).fit(X_2)
mean_vectors_background2 = kmeans_background2.cluster_centers_

In [ ]:
neg_log_likelihood_zebra_combined_2 = np.amin(
    np.dstack(
        [
            (
                sum(
                    (I_zebra[:, :, i] - mean_vectors_zebra2[n_cl, i]) ** 2
                    for i in range(3)
                )
            )
            for n_cl in range(len(mean_vectors_zebra2))
        ]
    ),
    2,
)

neg_log_likelihood_background_combined_2 = np.amin(
    np.dstack(
        [
            (
                sum(
                    (I_zebra[:, :, i] - mean_vectors_background2[n_cl, i]) ** 2
                    for i in range(3)
                )
            )
            for n_cl in range(len(mean_vectors_background2))
        ]
    ),
    2,
)

plt.figure()
plt.imshow(neg_log_likelihood_zebra_combined_2, cmap="gray", vmax=3000)
plt.title("neg_log_likelihood_zebra_combined_2")
plt.show()

plt.figure()
plt.imshow(neg_log_likelihood_background_combined_2, cmap="gray", vmax=1000)
plt.title("neg_log_likelihood_background_combined_2")
plt.show()


Q19: From these neg-log-likelihood images, segment the image by graph-cut using a new value of $\beta$ that gives you the best result. Comment on the result and the new value of $\beta$ that allowed you to obtain it. What about the new 5-class data attachment compared to the previous one?

**Your answer &#x270D;**

A19: The result is almost perfect. The only mistake is due to a grey rock behind the zebra's ear that is hard to differentiate from the animal, even visually.

We note that the value of beta is a lot smaller. This is due to the fact that we have 5 subclasses which implies that there is going to be more limits between classes and thus more penalization.

In the end we find that the new 5-class data attachment strategy modelizes more faithfully the data than the 2-class one. It allowed to have more precise sub-classes and thus reduce the overlap between the two classes.

In [ ]:
# Optimal beta value to be determined
beta = 10000

# Graph instantiation
g = maxflow.Graph[float]()

# Add the nodes. nodeids has the identifiers of the nodes in the grid.
nodeids = g.add_grid_nodes(I_zebra.shape[0:2])

# Add non-terminal edges with the same capacity.
g.add_grid_edges(nodeids, beta)

# Add the terminal edges.
g.add_grid_tedges(
    nodeids,
    neg_log_likelihood_background_combined_2,
    neg_log_likelihood_zebra_combined_2,
)

flow = g.maxflow()

print("Max Flow:", str(flow))
# Get the segments of the nodes in the grid.
sgm = g.get_grid_segments(
    nodeids
)  # Returns 1 if the pixel is on the drain side after calculation of the min cut, 0 if it is on the source side
im_bin = np.int_(np.logical_not(sgm))

affiche(im_bin, titre="Result for beta = " + str(beta))

plt.figure()
plt.imshow(im_bin)
plt.show()

plt.figure()
plt.imshow(display_segmentation_borders(I_zebra, im_bin))
plt.title("Red segmentation contours")
plt.show()